# Setting

In [135]:
singular.lib('sing.lib')
sage.libs.singular.function.lib("absfact.lib")
absFactorize = sage.libs.singular.function.singular_function("absFactorize")

#import Threefold_Terminal_Singularity as TTS
import sympy as sp
import copy
import collections
x,y,z,u = sp.symbols('x y z u')
variables = [x,y,z,u] 
type_symbol = type(x)
############### function ##############
#######################################
def compute_weight(f, weight):
    f = sp.Poly(f, gens = variables)
    return min([sum([weight[i]*degrees[i] for i in range(len(degrees))]) for degrees in f.monoms()])

def Zn_equivalent(element, index):
    return [num%index for num in element]

def monomials_list(f):
    '''
    Retrun a list consists of all  monomials of f
    
    Input parameter :
      f : a polynomial 

    Return : a list consists of all  monomials of f 

    Example : 
    >>> monomials_list(3*x**2-x*y*2+z**3*x) 
    [Poly(3*x**2, x, domain='ZZ'), Poly(-2*x*y, x, y, domain='ZZ'), Poly(x*z**3, x, z, domain='ZZ')]
    '''
    f = sp.Poly(f)
    return [c*sp.prod(x**k for x,k in zip(f.gens, mon)) for c, mon in zip(f.coeffs(), f.monoms())]


def homogeneous_part(f, n):
    '''
    Input parameter :
      f : a polynomial 
      n : an integer

    Return : 
      part : the homogeneous part of degree n in f

    Example : 
    >>> homogeneous_part(3*x**2-x*y*2+z**3*x,2) 
    3*x**2 - 2*x*y
    '''
    part = 0
    for term in monomials_list(sp.Poly(f)):
        if sp.Poly(term).total_degree()==n:
            part+=term
    return sp.simplify(part).as_expr()

def jet(f, n):
    '''
    Input parameter :
      f : a polynomial 
      n : an integer

    Return : 
      part : the part of f has degree no more than n

    Example : 
    >>> jet(3*x**2-x*y*2+z**3*x,2) 
    3*x**2-x*y*2
    '''
    part = 0
    for term in monomials_list(sp.Poly(f)):
        if sp.Poly(term).total_degree()<=n:
            part+=term
    return sp.simplify(part).as_expr()

def multiplicity(f):
    '''
    Input parameter :
      f : a polynomial 

    Return : 
      m : the multiplicity of f at origin where m = -1 if f = 0

    Example : 
    >>> multiplicity(3*x**2-x*y*2+z**3*x) 
    2
    '''
    if f == 0:
        return -1
    else:
        return min([sp.Poly(term).total_degree() for term in monomials_list(sp.Poly(f))])

def change_of_var(f, change_list):
    '''
    make a change of variables from var into new_var 
    for any (var, new_var) in change_list
    
    Input parameter :
      f : a polynomial
      change_list : [(var, new_var), ..., (var, new_var)]

    Return : f

    Example : 
    >>>change_of_var(sp.poly('3*x-2*y**2'),[(x, y-1), (y, x+1)])
    3*y - 2*(x + 1)**2 - 3
    '''
    n = len(change_list)
    var = [element[0] for element in change_list]
    new_var = [element[1] for element in change_list]
    replace_var = list(sp.symbols('u0:{}'.format(n)))

    for i in range(n):
        f = f.subs(var[i], replace_var[i])
    for i in range(n):
        f = f.subs(replace_var[i], new_var[i])
    return sp.simplify(sp.expand(f.as_expr()))

def change_of_var_with_group_action(f, change_list, index, group_action):
    '''
    make a change of variables from var into new_var 
    for any (var, new_var) in change_list
    
    Input parameter :
      f : a polynomial
      change_list : [(var, new_var), ..., (var, new_var)]

    Return : f

    Example : 
    >>>change_of_var(sp.poly('3*x-2*y**2'),[(x, y-1), (y, x+1)])
    3*y - 2*(x + 1)**2 - 3
    '''
    n = len(change_list)
    var = [element[0] for element in change_list]
    new_var = [element[1] for element in change_list]
    replace_var = list(sp.symbols('u0:{}'.format(n)))

    for i in range(n):
        f = f.subs(var[i], replace_var[i])
    for i in range(n):
        f = f.subs(replace_var[i], new_var[i])
        
    group_action = [compute_weight(term, group_action) for term in new_var]

    return sp.simplify(sp.expand(f.as_expr())), Zn_equivalent(group_action, index)
  
def Hessian_matrix(f):
    '''
    Input parameter :
      f : a polynomial

    Return : 
      Hess : the  Hessian matrix of f at origin

    Example : 
    >>> Hessian_matrix(sp.Poly('x**4 + 3*x**2 + 3*x*y + y**5 - 2*y**2 + z**6 - 4*z**3'))
    Matrix([[6, 3, 0], [3, -4, 0], [0, 0, 0]])
    '''
    var_list = f.gens
    Hess = sp.hessian(f, var_list, constraints=[])
    for i in var_list:
        Hess = Hess.subs(i,0)

    return Hess

def corank_Hessian_matrix(f):
    '''
    Input parameter :
      f : a polynomial

    Return : the corank of f's Hessian matrix at origin

    Example : 
    >>> corank_Hessian_Matrix(sp.Poly('x**4 + 3*x**2 + 3*x*y + y**5 - 2*y**2 + z**6 - 4*z**3'))
    1
    '''
    var_list=f.gens
    return len(var_list)-sp.Matrix.rank(Hessian_matrix(f))

def quadratic_form(f):
    '''
    Find out the symmetry matrix 
    corresponds to the quadratic form of degree 2 part of f.
    
    Input parameter :
      f : a polynomial

    Return : 
      Q : the symmetry matrix

    Example : 
    >>> quadratic_form(sp.Poly('x**4 + 3*x**2 + 3*x*y + y**5 - 2*y**2 + z**6 - 4*z**3'))
    Matrix([[  6, 3/2, 0],[3/2,  -4, 0],[  0,   0, 0]])
    '''
    Q = Hessian_matrix(f)/2
    return Q

def rearrange(v_list): 
    '''
    Input parameter :
      v_list : a list of some integer
      
    Return : 
      v : rearrange v_list such that all zeros are in the tail 

    Example : 
    >>> rearrange([1,0,-1,0,2])
    [2,1,-1,0,0]]
    '''
    v0 = [x for x in v_list if x==0]    
    v = sorted([x for x in v_list if x!=0])
    v.extend(v0)        
    return v

def quadratic_transform(f):
    '''
    Make a change of variables such that the degree 2 part is
    a diagonal form whose coefficients are all one 
    
    Input parameter :
      f : a polynomial
      
    Return : 
      f_new : a polynomial

    Example : 
    >>>quadratic_transform(sp.Poly('x**4 + x**2 + 3*x*y + z**5 + y**2 '))
    x**4 - 4*sqrt(5)*I*x**3*y/5 - 6*x**2*y**2/5 + 4*sqrt(5)*I*x*y**3/25 + y**4/25 + x**2 + y**2
    '''    
    Q = quadratic_form(f)
    v_eigenvec = Q.eigenvects()
    v_eigenvec = [list(x) for x in list(v_eigenvec)]

    eigenvalue0 = [x[0] for x in v_eigenvec]
    eigenvalue = rearrange(eigenvalue0)
    
    
    eigenv = []
    M = sp.Matrix()
    for i in eigenvalue:
        j = eigenvalue0.index(i)
        for vec in v_eigenvec[j][2]:
            eigenv.append(i)
            M = M.row_join(vec.normalized())
            
    diag = [1/sp.sqrt(x) if x != 0 else 1 for x in eigenv]
    D = sp.diag(*diag)
    #print(Q , D*M.T*Q*M*D)
    
    var = sp.Matrix(f.gens)
    new_var = (M*D)*var
    change = [(var[i],new_var[i]) for i in range(len(var))]
    f_new = sp.simplify(change_of_var(f, change))
    
    return f_new

def quadratic_transform_with_group_action(f, index, group_action):
    
    Q = quadratic_form(f)
    v_eigenvec = Q.eigenvects()
    v_eigenvec = [list(x) for x in list(v_eigenvec)]

    eigenvalue0 = [x[0] for x in v_eigenvec]
    eigenvalue = rearrange(eigenvalue0)
    
    
    eigenv = []
    M = sp.Matrix()
    for i in eigenvalue:
        j = eigenvalue0.index(i)
        for vec in v_eigenvec[j][2]:
            eigenv.append(i)
            M = M.row_join(vec.normalized())
            
    diag = [1/sp.sqrt(x) if x != 0 else 1 for x in eigenv]
    D = sp.diag(*diag)
    #print(Q , D*M.T*Q*M*D)
    
    var = sp.Matrix(f.gens)
    new_var = (M*D)*var
    
    change = [(var[i],new_var[i]) for i in range(len(var))]
    f_new = sp.simplify(change_of_var(f, change))
    
    group_action_new = [compute_weight(term[1], group_action) for term in change]
    
    return f_new, Zn_equivalent(group_action_new, index)


def complete_the_square(f, var):
    '''
    Complete the square f = a*(var-terms)**2 + g
    and make a change of variable
    such that f = var**2+new_g
    
    Input parameter :
      f : a polynomial
      var : a variable in f

    Return : new_f
      
    Example : 
    >>> complete_the_square(3*x**2+3*x*y-2*y*2-4*z*3, x)
    x**2 - 3*y**2/4 - 4*y - 12*z
    '''
    f_part = f - var**2 
    f_part_remainder = f_part.subs({var:0})
    f_part = f_part - f_part_remainder
    
    if f_part == 0:
        return f
    
    elif len(sp.Poly(f_part).gens) == 1:
        return f_part_remainder + var**2
    
    else:
        terms = sp.quo(f_part, var)/2
        f = f.subs(var, var-terms)
        f = f.expand()

        return sp.simplify(f).as_expr()

def complete_the_square_new(f, var, determinacy):
    '''
    Complete the square f = a*(var-terms)**2 + g with using finite determinacy theorem
    and make a change of variable
    such that f = var**2+new_g
    
    Input parameter :
      f : a polynomial
      var : a variable in f

    Return : new_f
      
    Example : 
    >>> complete_the_square(3*x**2+3*x*y-2*y*2-4*z*3, x)
    x**2 - 3*y**2/4 - 4*y - 12*z
    '''
    f_part = f - var**2 
    f_part_remainder = f_part.subs({var:0})
    f_part = f_part - f_part_remainder
    
    if f_part == 0:
        return f
    
    elif len(sp.Poly(f_part).gens) == 1:
        return f_part_remainder + var**2
    
    else:
        terms = sp.quo(f_part, var)/2
        f = f.subs(var, jet(var-terms, determinacy))
        f = jet(f.expand(), determinacy)

        return sp.simplify(f).as_expr()


def factor(f):
    ''' 
    factor the homogeneous polynomial
    and return all factors of it
    
    Input parameter :
      f : a polynomial

    Return : 
      factor_list : consists of all factors of f
      
    Example : 
    >>> factor(y*z**2+y**2*z+y**3)
    [y - z*(-1/2 - sqrt(3)*I/2), y - z*(-1/2 + sqrt(3)*I/2), y]
    '''
    f = sp.Poly(f) 
    var_list=f.gens
    factor_list = []
    for x in var_list:
        dic_roots = sp.roots(f, x)
    
        for a in dic_roots:
            for i  in range(dic_roots[a]):
                factor_list.append(sp.simplify(x-a))
                f = sp.quo(f,x-a)
    return factor_list

def find_linear_factor(f, n):
    ''' 
    find the linear factors of the degree n homogeneous part of f 
    
    Input parameter :
      f : a polynomial
      n : an integer

    Return : 
      num_roots : the number of the different linear factors
      factor_dic : the dictionary of all linear factors with multiplicity as value (type: collections.Counter)
      
    Examples : 
    >>> find_linear_factor(sp.Poly('y*z**2+y**2*z+y**3'), 2)
    (0, {})
    >>> find_linear_factor(sp.Poly('y*z**2+y**2*z+y**3'), 3)
    (3, Counter({y: 1, y - z*(-1/2 + sqrt(3)*I/2): 1, y - z*(-1/2 - sqrt(3)*I/2): 1}))
    '''
    f = homogeneous_part(f, n)
    if f == 0:
        num_roots, factor_dic = 0, {}
    else:
        factor_list = factor(f)
        factor_dic = collections.Counter(factor_list)
        num_roots = len(factor_dic)
        
    return num_roots

def cubic_form_lemma(f):
    ''' 
    determine  whether the cubic form f can be divisible by a square of a linear form or not
    
    Input parameter :
      f : a cubic form

    Return : 
        bool: True or False
        integer: the largest number of main linear factors
        polynomial: main linear factor
        polynomial: another linear factor
    '''
    f = sp.Poly(f) 
    var_list = f.gens
    
    assert len(var_list)<=3 and f.is_homogeneous, 'The input must be a cubic form!'
    
    partial_list = []
    for var in var_list:
        partial_list.append(f.diff(var))
    gcd = sp.gcd(partial_list)
    gcd_deg = gcd.total_degree()
    
    if gcd_deg > 0 and f.rem(gcd) == 0:
        if gcd_deg == 2:
            for var in var_list:
                gcd_diff = gcd.diff(var)
                if gcd_diff != 0:
                    return True, 3, gcd_diff/6, 0
                else:
                    pass
        else:
            l2 = f.quo(gcd**2)
            return True, 2, gcd, l2 
    else:
        return False, 0, 0, 0

    
def group_action_condition(singularity_type, index, group_action):
    if singularity_type == 'cA':
        if (index, group_action) in [(2, [0, 1, 1, 1]), (2, [1, 0, 1, 1])]:
            return 'cAx/2'
        elif (index, group_action) in [(4, [1, 3, 1, 2]), (4, [1, 3, 2, 1]), (4, [3, 1, 1, 2]), (4, [3, 1, 2, 1])]:
            return 'cAx/4'
        else:
            r = group_action[0]
            if gcd(index, r) == 1:
                if group_action  in [[r, index-r, 1, 0], [r, index-r, 0, 1]]:
                    return 'cA/'+str(index)
                return 'non-terminal'
            else:
                return 'non-terminal'            
    
    elif singularity_type == 'cD':
        if (index, group_action) in [(2, [1, 0, 1, 1]), (2, [1, 1, 0, 1]), (2, [1, 1, 1, 0])]:
            return 'cD/2'
        elif (index, group_action) in [(3, [0, 2, 1, 1]), (3, [0, 1, 2, 1]), (3, [0, 1, 1, 2])]:
            return 'cD/3'
        else:
            return 'non-terminal'  
    
    elif singularity_type == 'cE':
        if index == 2 and group_action == [1, 0, 1, 1]:
            return 'cE/2'
        else:
            return 'non-terminal'         
    
    else:
        return 'non-terminal'
    
    
############ main function ############
#######################################
def terminal_singularity_with_index_1(f0):
    f0 = sp.simplify(sp.Poly(f0).expand()).as_expr()
    var_list = sp.Poly(f0).gens
    
    f_sage = str(f0._sage_())     # however f_sage is a string
    P.<x, y, z, u> = PolynomialRing(CC)
    f_sage = P(f_sage)        # from string to sage expression
    
    # singular set up
    R = singular.ring(singular.complex, '(x,y,z,u)', 'ds')
    f_singular = singular(str(f_sage).replace('I', 'i')) # replace I by i

    
    f_hessian_singular = singular.jacob(singular.jacob(f_singular.jet(2)))
    f_rank =  f_hessian_singular.rank()
    f_corank = f_hessian_singular.nrows() - f_rank

    if f_rank >= 2:
        ## compute an upper bound of determinacy, i.e. estimated determinacy
        f_determinacy = f_singular.milnor() + 1

        #####------splitting_lemma-------#####
        f_i = quadratic_transform(sp.Poly(f0))
        for i in range(3, f_determinacy):
            f_i0 = f_i.copy() 

            for j in range(int(f_rank)):
                f_i = complete_the_square(f_i, var_list[j])

            if f_i == f_i0:
                break
        
        for j in range(int(f_rank)):
            f_i -= var_list[j]**2
        n = multiplicity(f_i) - 1
        return 'cA_%s' %n

    elif f_rank == 1:
        ## compute an upper bound of determinacy, i.e. estimated determinacy
        f_determinacy = f_singular.milnor() + 1

        #####------splitting_lemma-------#####
        f_i = quadratic_transform(sp.Poly(f0))
        
        for i in range(3, f_determinacy):
            f_i0 = f_i.copy() 

            for j in range(int(f_rank)):
                f_i = complete_the_square(f_i, var_list[j])

            if f_i == f_i0:
                break
        
        
        f_3_part = homogeneous_part(f_i, 3)

        if cubic_form_lemma(f_3_part)[0]:
            if cubic_form_lemma(f_3_part)[1] == 2:
                return 'cD>4'

            else:
                L1 = cubic_form_lemma(f_3_part)[2]

                # Row 1
                r1 = [L1.subs({var_list[1]:1,var_list[2]:0,var_list[3]:0}),
                                           L1.subs({var_list[1]:0,var_list[2]:1,var_list[3]:0}),
                                           L1.subs({var_list[1]:0,var_list[2]:0,var_list[3]:1})]

                linear_trans = sp.Matrix([r1])

                # choose standard basis
                I = sp.eye(3)
                for i in range(3):
                    if linear_trans.dot(I[i,:]) != 0:
                        j = i
                        break

                for i in range(3):
                    if i!=j:
                        linear_trans = linear_trans.col_join(I[i,:])
                


                linear_factor = list(linear_trans.inv()*sp.Matrix([[var_list[1]],[var_list[2]],[var_list[3]]]))
                f = change_of_var(f_i, [(var_list[1], linear_factor[0]), (var_list[2], linear_factor[1]), (var_list[3], linear_factor[2])])

                
                # f ~ x^2 + y^3 + y^2*g_1(y,z,u) + y*g_2(z,u) + g_3(z,u)
                f = jet(f, f_determinacy)
                for i in range(2):
                    g_1 = sp.quo(f - var_list[0]**2 - var_list[1]**3, var_list[1]**2)

                    if g_1 == 0:
                        break
                    else:

                        f = change_of_var(f, [(var_list[1], var_list[1] - g_1/3)])
                        f = jet(f, f_determinacy)
                        
                # f_4 = y*g(z,u) + h(z,u)
                f_4 = sp.rem(f - var_list[0]**2 - var_list[1]**3, var_list[1]**2)
                g = sp.quo(f_4, var_list[1])
                h = sp.rem(f_4, var_list[1])
                
                mult_g = multiplicity(g)
                mult_h = multiplicity(h)
                
                if mult_h == 4:
                    return 'cE_6'
                
                elif mult_g == 3:
                    return 'cE_7'
                
                elif mult_h == 5:
                    return 'cE_8'
                    
                else:
                    return 'non-terminal'

        else:
            return 'cD_4.'

    else:
        return 'non-terminal'

############################## index > 1 ###############################
def terminal_singularity_with_index_more_than_1(f0, index, group_action):
    f0 = sp.simplify(sp.Poly(f0).expand()).as_expr()
    var_list = sp.Poly(f0).gens
    
    f_sage = str(f0._sage_())     # however f_sage is a string
    P.<x, y, z, u> = PolynomialRing(CC)
    f_sage = P(f_sage)        # from string to sage expression
    
    # singular set up
    R = singular.ring(singular.complex, '(x,y,z,u)', 'ds')
    f_singular = singular(str(f_sage).replace('I', 'i')) # replace I by i

    f_hessian_singular = singular.jacob(singular.jacob(f_singular.jet(2)))
    f_rank =  f_hessian_singular.rank()
    f_corank = f_hessian_singular.nrows() - f_rank

    if f_rank >= 2:
        f_i, group_action = quadratic_transform_with_group_action(sp.Poly(f0), index, group_action)
        return group_action_condition('cA', index, group_action)

    elif f_rank == 1:
        ## compute an upper bound of determinacy, i.e. estimated determinacy + 1
        f_determinacy = f_singular.milnor() + 1

        #####------splitting_lemma-------#####
        f_i, group_action = quadratic_transform_with_group_action(sp.Poly(f0), index, group_action)
        for i in range(3, f_determinacy):
            f_i0 = f_i.copy() 

            for j in range(int(f_rank)):
                f_i = complete_the_square(f_i, var_list[j])

            if f_i == f_i0:
                break
        
        
        f_3_part = homogeneous_part(f_i, 3)
        
        if cubic_form_lemma(f_3_part)[0]:
            if cubic_form_lemma(f_3_part)[1] == 2:
                return group_action_condition('cD', index, group_action)

            else:

                L1 = cubic_form_lemma(f_3_part)[2]

                # Row 1
                r1 = [L1.subs({var_list[1]:1,var_list[2]:0,var_list[3]:0}),
                                           L1.subs({var_list[1]:0,var_list[2]:1,var_list[3]:0}),
                                           L1.subs({var_list[1]:0,var_list[2]:0,var_list[3]:1})]

                linear_trans = sp.Matrix([r1])

                # choose standard basis
                I = sp.eye(3)
                for i in range(3):
                    if linear_trans.dot(I[i,:]) != 0:
                        j = i
                        break

                for i in range(3):
                    if i!=j:
                        linear_trans = linear_trans.col_join(I[i,:])
                        
                linear_factor = list(linear_trans.inv()*sp.Matrix([[var_list[1]],[var_list[2]],[var_list[3]]]))
                f = change_of_var_with_group_action(f_i, [(var_list[1], linear_factor[0]), (var_list[2], linear_factor[1])
                                                          , (var_list[3], linear_factor[2])], index, group_action)


                # f ~ x^2 + y^3 + y^2*g_1(y,z,u) + y*g_2(z,u) + g_3(z,u)
                f = jet(f, f_determinacy)

                for i in range(2):
                    g_1 = sp.quo(f - var_list[0]**2 - var_list[1]**3, var_list[1]**2)

                    if g_1 == 0:
                        break
                    else:
                        f = change_of_var(f, [(var_list[1], var_list[1] - g_1/3)])
                        f = jet(f, f_determinacy)
                        
                # f_4 = y*g(z,u) + h(z,u)
                f_4 = sp.rem(f - var_list[0]**2 - var_list[1]**3, var_list[1]**2)
                
                g = sp.quo(f_4, var_list[1])
                h = sp.rem(f_4, var_list[1])

                mult_g = multiplicity(g)
                mult_h = multiplicity(h)
                
                if mult_h == 4 or 5 and mult_g == 3:
                    return group_action_condition('cE', index, group_action)
                    
                else:
                    return 'non-terminal'

        else:
            return group_action_condition('cD', index, group_action)

    else:
        return 'non-terminal'

In [136]:
def singularity(f0):
    '''
    make a change of variables w.r.t its singular point

    Input parameter :
      f : a polynomial
    
    Return :
      If f doesn't have any singularities, return empty list []
      On the other hand, return the information in this form:
          [(transformed f, singularity , 'whether it is an isolated singularity or not'),...,()]
    
    Example:
    >>> change_of_var_singularity(sp.Poly('u**8 + u**6 + u*z**5 + x**2 + y**3'))
    [(u**8 + u**6 + u*z**5 + x**2 + y**3, {u: 0, x: 0, y: 0, z: 0}, 'Isolated singularity')]
    '''
    
    f = sp.Poly(f0)
    var_list = f.gens
    n = len(var_list)
    f_diff = [sp.diff(f, x) for x in var_list]
    f_diff.append(f)

    sg_dic = sp.solve(f_diff, var_list, dict = True)
    
    sg_list = [dic.items() for dic in sg_dic] 
    m = len(sg_list)
    sg_index_set = []
    
    
    for i in range(m):
        solution1 = sg_list[i]
        l1 = len(solution1)
        check = 1

        for j in range(m):
            if i == j:
                pass
            else:
                solution2 = sg_list[j]
                l2 = len(solution2)
                if l1 == l2 and all(elem in solution1 for elem in solution2):
                    if i > j:
                        check = 0
                elif l1 > l2 and all(elem in solution1 for elem in solution2):
                    check = 0
        if check == 1:
            sg_index_set.append(i)
            
    sg_dic = [sg_dic[i] for i in sg_index_set]
    
    if len(sg_dic) == 0: 
        return []
    
    else:
        sg_transform = []
        for solution in sg_dic:
            if len(solution) == n:
                if is_origin(solution):
                    sg_transform.append([solution, 'Isolated singularity on the origin'])
                else:
                    sg_transform.append([solution, 'Isolated singularity but not on the origin'])
            else:
                sg_transform.append([solution, 'Non-isolated singularity'])
                
        return sg_transform
    
def is_origin(dic):
    for key in dic.keys():
        if dic[key] != 0:
            return False
    return True

def only_has_isolated_singularities(sg_transform):
    sg_type = [sg[1] for sg in sg_transform]
    if 'Non-isolated singularity' in sg_type:
        return False
    else:
        return True

# terminal + nonsingular
def is_terminal(f, index, group_action):
    var_list = sp.Poly(f).gens
    sg_transform = singularity(f)
    if len(sg_transform) == 0:
        return 'non-singular'
    else:
        if only_has_isolated_singularities(sg_transform):
            sg_list = []

            for solution, sg_type in sg_transform:
                if sg_type == 'Isolated singularity but not on the origin' or index == 1:
                    transform = [(var_list[i], var_list[i] + solution[var_list[i]]) for i in range(len(list(var_list)))]
                    f_sg = change_of_var(f,transform)
                    sg_list.append(terminal_singularity_with_index_1(f_sg))

                else:
                    sg_list.append(terminal_singularity_with_index_more_than_1(f ,index, group_action))

            print('it only has isolated singularities which are: ')
            for i in range(len(sg_list)):      
                print('({},{})'.format(sg_transform[i][0], sg_list[i]))
            if 'non-terminal' in sg_list:
                return 'non-terminal'
            else:

                return 'terminal'

        else:
            print('the singularities are: ')
            for i in range(len(sg_transform)):      
                print('({},{})'.format(sg_transform[i][0], sg_transform[i][1]))
            
            return 'non-isolated'
    
def wt_homogeneous_part(f, n, weight):
    part = 0
    for term in monomials_list(sp.Poly(f)):
        if compute_weight(sp.Poly(term), weight) == n:
            part += term
    return sp.simplify(part).as_expr()

def weighted_blowup(f, weight):
    f_wt_blowup = []
    f_weight = compute_weight(f, weight)
    
    for i in range(4):
        f_sp = f.copy()

        for j in range(4):
            if j==i:
                f_sp = f_sp.subs(variables[j],variables[j]**weight[j])
            else:
                f_sp = f_sp.subs(variables[j],variables[j]*variables[i]**weight[j])

        f_sp = sp.simplify(sp.factor(sp.factor(f_sp)/variables[i]**f_weight))
        f_wt_blowup.append(f_sp)
    return f_wt_blowup

def is_irreducible(f_singular):
    S = singular.absFactorize(f_singular);
    singular.setring(S);
    L = singular('absolute_factors')
    if L[4] == 1:
        return True
    else:
        return False
    
def compute_all_divisorial_contraction(f, admissble_weights):
    divisorial_contraction_list = []

    for weight in admissble_weights:
        divisorial_contraction = True

        print('weight: {}'.format(weight))
        f_wt_blowup = weighted_blowup(f, weight)

        ### 取出exceptional
        for i in range(len(f_wt_blowup)):
            f_wt = f_wt_blowup[i]
            print('the strict transform on U_{}: {}'.format(variables[i],f_wt))

            index = weight[i]
            quotient_type = copy.copy(weight)
            quotient_type[i] = -1
            group_action = Zn_equivalent(quotient_type, index)

            wt_f_wt = compute_weight(f_wt, weight)
            
            if wt_f_wt == 0:
                print('non_singular')
            else:
                #f_except = sp.Poly(wt_homogeneous_part(f_wt, compute_weight(f_wt, weight), weight))
                f_except = f_wt.subs(variables[i], 0)
                #print type(f_except)
                f_except = f_except.as_expr()

                if type(f_except) == type_symbol:
                    print('it is non-singular.')
                else:
                    # transform to the type in singular
                    R = singular.ring(0, '(x,y,z,u)', 'dp')
                    f_except_singular = singular(str(f_except))

                    if is_irreducible(f_except_singular):
                        if is_terminal(f_wt, index, group_action) == 'terminal':
                            print('it is terminal!')

                        elif is_terminal(f_wt, index, group_action) == 'non-singular':
                            print('it is non-singular.')

                        elif is_terminal(f_wt, index, group_action) == 'non-isolated':
                            divisorial_contraction = False
                            print('there are non-isolated singularities')

                        else:
                            divisorial_contraction = False
                            print('it is not terminal!')

                    else:
                        divisorial_contraction = False
                        print('the exceptional set is defined by {} which is not irreducible!'.format(f_except))
            print('\n')


        if divisorial_contraction:
            divisorial_contraction_list.append(weight)
            print('{} defines a divisorial contraction to the germ of {} \n').format(weight,f_string)

    print('The following weights give divisorial contractions to the germ of {} \n').format(f_string)
    for weight in divisorial_contraction_list:
        print('{}').format(weight) 

# Determine All Divisorial Contractions to points

In [137]:
f_string = 'x**2+y**3+z**4+u**6'
admissble_weights = [[3,2,2,1],[3,2,1,1],[2,2,1,1]]

f = sp.Poly(f_string)
compute_all_divisorial_contraction(f, admissble_weights)

weight: [3, 2, 2, 1]
the strict transform on U_x: u**6 + x**2*z**4 + y**3 + 1
non_singular


the strict transform on U_y: u**6 + x**2 + y**2*z**4 + 1
non_singular


the strict transform on U_z: u**6 + x**2 + y**3 + z**2
it only has isolated singularities which are: 
({u: 0, x: 0, z: 0, y: 0},cA/2)
it is terminal!


the strict transform on U_u: u**2*z**4 + x**2 + y**3 + 1
non_singular


[3, 2, 2, 1] defines a divisorial contraction to the germ of x**2+y**3+z**4+u**6 

weight: [3, 2, 1, 1]
the strict transform on U_x: u**6*x**2 + x**2*y**3 + x**2 + z**4
the exceptional set is defined by z**4 which is not irreducible!


the strict transform on U_y: u**6*y**2 + x**2*y**2 + y**2 + z**4
the exceptional set is defined by z**4 which is not irreducible!


the strict transform on U_z: u**6*z**2 + x**2*z**2 + y**3*z**2 + 1
non_singular


the strict transform on U_u: u**2*x**2 + u**2*y**3 + u**2 + z**4
the exceptional set is defined by z**4 which is not irreducible!


weight: [2, 2, 1, 1]
the stri

### Define it into a function

In [260]:
def determine_all_divisorial_contraction(f ,admissble_weights):
    # cE_6 = [[6,4,3,1],[4,3,2,1],[3,2,2,1],[2,2,1,1]]
    divisorial_contraction_list = []


    for weight in admissble_weights:
        divisorial_contraction = True
        f_wt_blowup = weighted_blowup(f, weight)

        ### 取出exceptional
        for i in range(len(f_wt_blowup)):
            f_wt = f_wt_blowup[i]
            #print('the strict transform on U_{}: {}'.format(variables[i],f_wt))

            index = weight[i]
            quotient_type = copy.copy(weight)
            quotient_type[i] = -1
            group_action = Zn_equivalent(quotient_type, index)

            wt_f_wt = compute_weight(f_wt, weight)
            if wt_f_wt == 0:
                pass
                #print('non_singular')
            else:
                f_except = f_wt.subs(variables[i], 0)
                f_except = f_except.as_expr()

                if type(f_except) == type_symbol:
                    pass
                    #print('non_singular')
                else:
                    # transform to the type in singular
                    R = singular.ring(0, '(x,y,z,u)', 'dp')
                    f_except_singular = singular(str(f_except))

                    if is_irreducible(f_except_singular):
                        if is_terminal(f_wt, index, group_action):
                            pass
                        else:
                            divisorial_contraction = False

                    else:
                        divisorial_contraction = False

        if divisorial_contraction:
            divisorial_contraction_list.append(weight)
    return divisorial_contraction_list


In [262]:
divisorial_contraction_list = determine_all_divisorial_contraction(f ,admissble_weights)
if len(divisorial_contraction_list) == 0:
    print('there is no divisorial contractions!')
else:
    for weight in divisorial_contraction_list:
        print('{} defines a divisorial contraction to the germ of {} \n\n').format(weight,f_string)

there is no divisorial contractions!


In [126]:

index = 2
group_action = [1,1,1,1]
if (index, group_action) in [(2, [0, 1, 1, 1]), (2, [1, 0, 1, 1])]:
    print('幹')